In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-08 14:57:23 Didis-MacBook-Pro.local metapub.config[39330] WARNING NCBI_API_KEY was not set.


10.1016/j.conb.2007.07.003
17707635


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [4]:
# merge all search results, fill in elements missed, remove duplciations based on identifiers in the potential related literature
def fill_in_identifiers(input, output_path):
    # combine all results
    df = pd.DataFrame()
    for search_result in input:
        df_single = pd.read_csv(search_result, header=None, sep = ",")
        df = df.append(df_single, ignore_index=True, sort=False)
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_url", "full_text_source", "pdf_url", "pdf_source"]
    print(df.shape)
    
    # fill in elements that are missing
    # DOI -> PMID
    # PMID -> DOI
    # PMID -> PMCID
    # PMCID -> PMID
    for ind in df.index:
        # doi, pmid
        if df["DOI"][ind] == df["DOI"][ind]:
            doi = str(df["DOI"][ind]).strip()
            if df["PMID"][ind] == df["PMID"][ind]:
                pmid = df["PMID"][ind]
            else:
                pmid = plib.doi2pmid(doi)
        elif df["PMID"][ind] == df["PMID"][ind]:
            doi = plib.pmid2doi(df["PMID"][ind])
            pmid = df["PMID"][ind]
        elif df["PMCID"][ind] == df["PMCID"][ind]:
            doi, pmid = plib.pmcid2doi_pmid(df["PMCID"][ind])

        # pmcid
        if df["PMCID"][ind] == df["PMCID"][ind]:
            pmcid = df["PMCID"][ind]
        elif df["PMID"][ind] == df["PMID"][ind]:
            doi_1, pmcid = plib.pmid2doi_pmcid(df["PMID"][ind])
        else:
            pmcid = np.nan

        title = str(df["Title"][ind]).strip().lower()

        # full_text_link
        if pmcid == pmcid:
            full_text_link = "https://www.ncbi.nlm.nih.gov/pmc/articles/" + pmcid + "/"
        elif doi == doi:
            full_text_link = plib.get_final_redirected_url(str("https://doi.org/" + doi).strip())
        elif df["full_text_url"][ind] == df["full_text_url"][ind]:
            full_text_link = plib.get_final_redirected_url(df["full_text_url"][ind])
        else:
            full_text_link = np.nan

        # pdf_link
        if df["pdf_url"][ind] == df["pdf_url"][ind]:
            pdf_link = plib.get_final_redirected_url(df["pdf_url"][ind])
        else:
            pdf_link = np.nan
    
    columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
    row = {
        "DOI": [doi],
        "PMID": [pmid],
        "PMCID": [pmcid],
        "Title": [title],
        "full_text_link": [full_text_link],
        "pdf_link": [pdf_link],
    }

    if not plib.add_row_to_csv(output_path, row, columns):
        print("Error detected when adding a row to csv!")

    print(ind)
# --------------------start of test code--------------------
# gos = fpath.poten_litera_gs_processed_step2
# wos = fpath.poten_litera_wos_processed
# pubmed = fpath.poten_litera_pubmed_processed
# eupmc = fpath.poten_litera_eupmc_processed
# input = [gos, wos, pubmed, eupmc]
# output_path = fpath.poten_litera
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# fill_in_identifiers(input, output_path)
# fill_in_identifiers(input, output, start, end)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------

In [ ]:
def merge_remove_dupli(input_path, output_path): 
    df = pd.read_csv(input_path, header=None, sep = ",")

    # remove all duplicates
    df = df.drop_duplicates(subset=['DOI'])
    df = df.drop_duplicates(subset=['PMID'])
    df = df.drop_duplicates(subset=['PMCID'])
    print("Duplication in the potential related literature removed.")

    print("Found " + len(df) + " potential related literature in total.")

    # write the results into our output file
    df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
    df.to_csv(output_path, header=True, index=False)
# --------------------start of test code--------------------
# source_path = fpath.poten_litera
# output_path = fpath.filtered_poten_litra
# plib.clear_file(output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(source_path, header=None, sep=',')
# df.columns = ["DOI", "PMID", "PMCID", "Title", "full_text_link", "pdf_link"]
# print(df.head(3))
# print(df.shape)
# # 
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# # merge all search results
# identifier = ["DOI", "PMID", "PMCID"]
# merge_remove_dupli(input_path, output_path)
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# df = pd.read_csv(output_path, header=None, sep=',')
# print(df.head(3))
# ---------------------end of test code---------------------